In [2]:
import numpy as np
import timeit

In [3]:
def get_activation_paths(polytopes):
  n_samples = polytopes[0].shape[0]
  #reverse to save on calculation time using smaller later layers
  polytope_r = polytopes[::-1]
  place = [1]
  for layer in polytope_r:
    p = place[-1]
    place.append(p*layer.shape[1])

  #set error code
  #this value ensures final output will be negative for all nodes and code will run
  err = np.array([-place[-1]])
  #get paths values
  paths = [np.zeros(1, dtype=int) for n in range(n_samples)]
  for i, layer in enumerate(polytope_r):
    idx = layer*np.arange(layer.shape[1])*place[i]
    temp_paths = [None for n in range(n_samples)]
    for j in range(n_samples):
      active_nodes = idx[j, layer[j,:]>0]
      if len(active_nodes) == 0:
        temp_paths[j] = err
      else: 
        temp_paths[j] = np.concatenate([p + active_nodes for p in paths[j]])
        
    paths = temp_paths
    #print(paths)
    
  #convert to binary
  activation_paths = np.zeros((n_samples, place[-1]))
  for i, p in enumerate(paths):
    #if error occured, return None
    if any(p < 0): activation_paths[i] = -1
    else: activation_paths[i, p] = 1

  return activation_paths

In [4]:
def get_activation_paths_2(polytopes):
  n_samples = polytopes[0].shape[0]
  #reverse to match old algorithm on calculation time using smaller later layers
  #polytopes = polytopes[::-1]
  place = [1]
  for layer in polytopes:
    p = place[-1]
    place.append(p*layer.shape[1])
  paths = np.zeros((n_samples, place[-1]), dtype=int)
  for i, layer in enumerate(polytopes):
    #expand layer to match place
    tiled_cols = [np.tile(layer[:,col], reps = [place[i], 1]) for col in range(layer.shape[1])]
    #combine into single layer of binary activations
    tiled_layer = np.concatenate(tiled_cols, axis=0).T
    #print(f"Expanded: {tiled_layer.shape}")
    ntiles = int(place[-1]/tiled_layer.shape[1])
    #add activated nodes to paths
    paths = paths + np.tile(tiled_layer, reps=[1, ntiles])
  
  #select only paths which were activated at every step
  activation_paths = (paths == len(polytopes)) * 1
  #activation_paths[np.all(activation_paths == 0, axis = 1)] = -1
  return activation_paths

In [5]:
#small network

network_shape = [5, 5, 2]
n_samples = 10000

#create a random string of binary numbers for "activations"
polytope = [np.rint(np.random.rand(n_samples, l)).astype(int) for l in network_shape]

for p in polytope:
  print(p.shape)

timer1 = timeit.Timer(lambda: get_activation_paths(polytope))
timer2 = timeit.Timer(lambda: get_activation_paths_2(polytope))

print(f"Runtime for algorithm 1: {timer1.timeit(number=1)}")
print(f"Runtime for algorithm 2: {timer2.timeit(number=1)}")

(10000, 5)
(10000, 5)
(10000, 2)
Runtime for algorithm 1: 0.33779384500030574
Runtime for algorithm 2: 0.022430705999795464


In [6]:
#medium network
network_shape = [10, 10, 2]
n_samples = 10000

#create a random string of binary numbers for "activations"
polytope = [np.rint(np.random.rand(n_samples, l)).astype(int) for l in network_shape]

for p in polytope:
  print(p.shape)

timer1 = timeit.Timer(lambda: get_activation_paths(polytope))
timer2 = timeit.Timer(lambda: get_activation_paths_2(polytope))

print(f"Runtime for algorithm 1: {timer1.timeit(number=1)}")
print(f"Runtime for algorithm 2: {timer2.timeit(number=1)}")

(10000, 10)
(10000, 10)
(10000, 2)
Runtime for algorithm 1: 0.42250384599992685
Runtime for algorithm 2: 0.06508385799997995


In [7]:
#kdcnn dense network
network_shape = [20, 20, 20, 10]
n_samples = 1000

#create a random string of binary numbers for "activations"
polytope = [np.rint(np.random.rand(n_samples, l)).astype(int) for l in network_shape]

for p in polytope:
  print(p.shape)

timer1 = timeit.Timer(lambda: get_activation_paths(polytope))
timer2 = timeit.Timer(lambda: get_activation_paths_2(polytope))

print(f"Runtime for algorithm 1: {timer1.timeit(number=1)}")
print(f"Runtime for algorithm 2: {timer2.timeit(number=1)}")

(1000, 20)
(1000, 20)
(1000, 20)
(1000, 10)
Runtime for algorithm 1: 1.594338377999975
Runtime for algorithm 2: 2.744605806999971


In [8]:
n_samples = 5
polytope0 = [np.zeros((n_samples, l), dtype=int) for l in network_shape]
polytope1 = [np.ones((n_samples, l), dtype=int) for l in network_shape]

print(np.unique(get_activation_paths(polytope0), axis=1))
print(np.unique(get_activation_paths_2(polytope0), axis=1))
print(np.unique(get_activation_paths(polytope1), axis=1))
print(np.unique(get_activation_paths_2(polytope1), axis=1))

[[-1.]
 [-1.]
 [-1.]
 [-1.]
 [-1.]]
[[0]
 [0]
 [0]
 [0]
 [0]]
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]]
[[1]
 [1]
 [1]
 [1]
 [1]]
